In [18]:
import pandas as pd
from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.StringIO(uploaded['synthetic.csv'].decode('utf-8')))
df

Saving synthetic.csv to synthetic.csv


,Attr_A,Attr_B,Attr_C,Attr_D,Attr_E,Attr_F,Attr_G,Attr_H,Attr_I,Attr_J,Attr_K,Attr_L,Attr_M,Attr_N,Class
0,12.478064,14.784992,1247.541877,100.962061,52.462177,1089.398211,10.575834,8.375407,10.288159,110.746551,994.367610,9.069350,1027.953917,109.672758,1
1,8.264345,8.854181,1389.686814,99.526529,65.106526,1186.523399,9.500485,10.088058,9.371983,78.210274,943.089589,9.988919,1120.317724,83.498764,3
2,9.875571,13.292442,779.077401,123.993772,104.699796,1201.722480,9.545266,14.266238,9.703551,86.252483,1082.989190,10.084217,970.953682,93.557046,2
3,9.207661,9.346913,965.468523,89.176009,102.628284,743.913507,9.777953,11.613946,8.912059,96.727873,812.800511,8.621781,947.207195,120.890054,3
4,8.863842,12.542969,1096.386230,106.595385,131.813380,883.059615,10.092974,13.556029,11.649982,21.566576,971.083175,10.072271,1007.583900,149.511979,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,8.537905,9.384823,1327.744849,105.043442,109.078735,1160.837366,10.054976,6.461691,9.718114,146.371553,1057.578998,9.504676,896.830168,50.119576,1
2294,10.727881,12.758686,1017.495424,103.388471,65.953337,955.265960,11.040854,5.930806,9.690326,92.318732,960.311986,9.262439,1158.151512,110.854719,1
2295,9.501612,12.449207,934.067397,94.681564,108.414931,1097.411669,11.379111,12.178924,9.161475,63.980205,972.994510,7.685836,996.569164,89.850887,0
2296,11.581467,12.024253,1247.894013,95.732086,85.265126,1012.536672,9.972115,9.001799,11.909924,128.966934,927.437658,8.906923,891.006210,93.561794,1


In [ ]:
print(df["Attr_A"][0])

12.478064498650886


In [39]:
import math as m
def entropie(df):
  sum=0
  nbElements = df.shape [0] 
  series = df['Class'].value_counts()
  for i in series:
    proba = i/nbElements
    sum -= proba * m.log2(proba)
  return -sum

In [38]:
def Gain(df, partition, P):
  taille = df.shape[0]
  gain = entropie(df)
  for i in range(P):
    gain = gain - (len(partition[i])/taille) * entropie(partition[i])
  return gain

In [95]:
'''def gain(df, a, split_value) : 
  g = 0
  partitions = [None,None]
  sorted_data = df.sort_values(by = a, ignore_index=True)
  for i in range(0,sorted_data.shape[0]):
    if sorted_data[a][i] == split_value : 
      partitions[0] = sorted_data[:i]
      partitions[1] = sorted_data[i:]
      g = Gain(df,partitions,len(partitions)) 
      break
  return (-g,partitions)
  '''
def gain(df, attr, split_value):
  e = entropie(df)
  gain = 0
  partitions = [[], []]
  sorted_data = df.sort_values(by = attr).reset_index(drop = True)

  for _, row in sorted_data.iterrows():
      if (row[attr] < split_value):
          partitions[0] += [row]
      else:
          partitions[1] += [row]

  gain = e
  for i in range(2):
      if partitions[i]:
          gain -= (len(partitions[i]) / df.shape[0]) * entropie(pd.DataFrame(partitions[i]))

  return (-gain, partitions)

In [11]:
class Noeud(object):
  def __init__(self,prediction, attribut=None, nfd = None, nfg =None,split_value = None):
    self.attribut = attribut
    self.right= nfd
    self.left = nfg
    self.prediction = prediction
    self.split_value = split_value
  def feuille(self):
    return self.left is None and self.right is None
  def resultat(self):
    res = ''
    for att in self.prediction.index.tolist():
      res += "Classe {}: Count {} ".format(att,self.prediction[att])
    return res

In [104]:
def Decision_Tree(df,attribut_restant,profondeur,seuil):
  g = 0
  print(attribut_restant)
  pred = df["Class"].value_counts()
  for a in attribut_restant:
    for i in range(3):
      if(i==0):
        quartile = df.quantile(q=0.25, axis=0, numeric_only=True, interpolation='linear')[a]
      elif(i==1):
        quartile = df.quantile(q=0.5, axis=0, numeric_only=True, interpolation='linear')[a]
      else:
        quartile = df.quantile(q=0.75, axis=0, numeric_only=True, interpolation='linear')[a]
      (gainTemp,partitionsTemp) = gain(df,a,quartile)
      if(gainTemp > g):
        g = gainTemp
        attribut = a
        split_value = quartile
        partitions = partitionsTemp
  attribut_restant.remove(attribut)
  if attribut_restant == None or profondeur > seuil:
    return Noeud(pred)
  branche_gauche = Decision_Tree(partitions[0],attribut_restant,profondeur + 1,seuil)
  branche_droite = Decision_Tree(partitions[1],attribut_restant,profondeur + 1,seuil)
  return Noeud(pred,attribut,branche_droite,branche_gauche,split_value)

In [83]:
def print_tree(node, spacing=''):
  if node is None:
    return
  if node.feuille():
    print(spacing + node.resultat())
    return
  print('{}[Attribute: {} Split value: {}]'.format(spacing,node.attribut,node.split_value))

  print(spacing + '> True')
  print_tree(node.left, spacing + '-')

  print(spacing + '> False')
  print_tree(node.right, spacing + '-')
  return


In [105]:
r = df.columns.tolist()
r.remove("Class")
print_tree(Decision_Tree(df,r,0,4))

['Attr_A', 'Attr_B', 'Attr_C', 'Attr_D', 'Attr_E', 'Attr_F', 'Attr_G', 'Attr_H', 'Attr_I', 'Attr_J', 'Attr_K', 'Attr_L', 'Attr_M', 'Attr_N']
['Attr_A', 'Attr_C', 'Attr_D', 'Attr_E', 'Attr_F', 'Attr_G', 'Attr_H', 'Attr_I', 'Attr_J', 'Attr_K', 'Attr_L', 'Attr_M', 'Attr_N']


TypeError: ignored

In [42]:
lst = []
i=0
for e in df.columns: 
  if(e != 'Class'):
    split = df.quantile(q=0.25, axis=0, numeric_only=True, interpolation='linear')[i]
    (g,p)= gain(df,e,split)
    lst.append(g)
  i += 1
lst

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [65]:
r = df.columns.tolist()
r.remove('Class')
print(r)

['Attr_A', 'Attr_B', 'Attr_C', 'Attr_D', 'Attr_E', 'Attr_F', 'Attr_G', 'Attr_H', 'Attr_I', 'Attr_J', 'Attr_K', 'Attr_L', 'Attr_M', 'Attr_N']
